## Homography Crop

This code given an image -thermal baby image- and a its corresponding visible counterpart by selecting points in the image is able to craft a homography matrix and transform all visible images in a folder accordingly. This notebook specifically was made to convert those images to 480 × 640.

### **Disclaimer: This code should be executed with the following things in mind.**
- A good homography matrix, the best possible one, compute as many points as possible, based on two extremely clear images.
- It should be executed twice, one for each camera, separating images in folders depending on the camera.

In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob

In [20]:
# Provide file paths
# !!!! This are the images that will be used for the homography matrix creation
# We should work twice, one per thermal camera

thermal_path = "/Users/marino/Downloads/HM20240814214018.jpeg" # for the thermal image for the homography matrix
visible_path = "/Users/marino/Downloads/visible/HM20240814214018_VIS.jpeg" # for the visible image for the homography matrix
matrix_path = "/Users/marino/Downloads/homography_matrix.txt" # to save the homography matrix (if good)

visible_folder = "/Users/marino/Downloads/visible" # for the visible images, to warp/transform
output_folder = "/Users/marino/Downloads/aligned" # for the aligned images, to save the warped images

Point selection

In [15]:
# Load images
visible_img = cv2.imread(visible_path)
thermal_img = cv2.imread(thermal_path)

visible_pts = []
thermal_pts = []

# --- Mouse click handler ---
def click_points(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        img, pts, name = param
        pts.append([x, y])
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        cv2.putText(img, f'{len(pts)}', (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
        cv2.imshow(name, img)

# Clone images for display
visible_clone = visible_img.copy()
thermal_clone = thermal_img.copy()

# Collect points from visible image
cv2.imshow("Visible Image - Click 4+ keypoints", visible_clone)
cv2.setMouseCallback("Visible Image - Click 4+ keypoints", click_points, [visible_clone, visible_pts, "Visible Image - Click 4+ keypoints"])
cv2.waitKey(0)
cv2.destroyAllWindows()

# Collect points from thermal image
cv2.imshow("Thermal Image - Click same keypoints in same order", thermal_clone)
cv2.setMouseCallback("Thermal Image - Click same keypoints in same order", click_points, [thermal_clone, thermal_pts, "Thermal Image - Click same keypoints in same order"])
cv2.waitKey(0)
cv2.destroyAllWindows()

# Convert to numpy arrays
src_pts = np.array(visible_pts, dtype=np.float32)
dst_pts = np.array(thermal_pts, dtype=np.float32)

# Compute homography matrix
H, status = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)

# Warp visible image
aligned_visible = cv2.warpPerspective(visible_img, H, (thermal_img.shape[1], thermal_img.shape[0]))


In [16]:
# Save aligned image
cv2.imwrite("/Users/marino/Downloads/manual_aligned_visible.jpg", aligned_visible)

# Overlay for sanity check
blended = cv2.addWeighted(cv2.cvtColor(aligned_visible, cv2.COLOR_BGR2RGB), 0.5,
                          cv2.cvtColor(thermal_img, cv2.COLOR_BGR2RGB), 0.5, 0)

cv2.imshow("Overlay: Aligned Visible + Thermal", cv2.cvtColor(blended, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()

print(H)

[[ 3.53613139e-01  1.21078978e-02 -2.16383130e+02]
 [ 5.28040545e-02  3.55534802e-01 -2.64656555e+02]
 [ 1.61694816e-05  5.70241601e-05  1.00000000e+00]]


Saving the homography matrix as a txt to reuse if good, just a failsafe.

In [17]:
def save_homography_matrix(H, filename):
    """
    Saves the homography matrix H to a .txt file.

    Parameters:
        H (np.ndarray): 3x3 Homography matrix.
        filename (str): Output text file path.

    """
    np.savetxt(filename, H, fmt='%.8f', delimiter=',')
    print(f"Homography matrix saved to {filename}")

# Save the homography matrix to a text file
save_homography_matrix(H, matrix_path)

Homography matrix saved to /Users/marino/Downloads/homography_matrix.txt


Given homography matrix: warp images from folder

In [18]:
def warp_images_from_folder(visible_folder, H, target_shape, output_dir, extensions=[".jpg", ".png", ".jpeg"]):
    """
    Applies a homography matrix to all images in a folder.

    Parameters:
        visible_folder (str): Path to the folder with visible images.
        H (np.ndarray): 3x3 Homography matrix.
        target_shape (tuple): Shape (height, width) to warp images to (from the thermal image).
        output_dir (str): Directory to save warped images.
        extensions (list): List of valid image file extensions.

    Returns:
        None
    """
    os.makedirs(output_dir, exist_ok=True)

    # Get all image files in the folder
    image_paths = []
    for ext in extensions:
        image_paths.extend(glob(os.path.join(visible_folder, f"*{ext}")))

    if not image_paths:
        print("No images found in the folder.")
        return

    for image_path in image_paths:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Could not read {image_path}")
            continue

        warped_img = cv2.warpPerspective(img, H, (target_shape[1], target_shape[0]))
        filename = os.path.basename(image_path)
        save_path = os.path.join(output_dir, f"aligned_{filename}")
        cv2.imwrite(save_path, warped_img)
        print(f"Saved aligned image to {save_path}")


In [21]:

target_shape = thermal_img.shape[:2]  # (height, width)

warp_images_from_folder(visible_folder, H, target_shape, output_folder)

Saved aligned image to /Users/marino/Downloads/aligned/aligned_HM20240814214018_VIS.jpeg
Saved aligned image to /Users/marino/Downloads/aligned/aligned_HM20240814213403_VIS.jpeg
Saved aligned image to /Users/marino/Downloads/aligned/aligned_HM20240814213710_VIS.jpeg
Saved aligned image to /Users/marino/Downloads/aligned/aligned_HM20240814213507_VIS.jpeg
Saved aligned image to /Users/marino/Downloads/aligned/aligned_HM20240814213609_VIS.jpeg
Saved aligned image to /Users/marino/Downloads/aligned/aligned_HM20240814213916_VIS.jpeg
Saved aligned image to /Users/marino/Downloads/aligned/aligned_HM20240814213814_VIS.jpeg
Saved aligned image to /Users/marino/Downloads/aligned/aligned_HM20240814214120_VIS.jpeg
